In [1]:
%matplotlib inline 

# Caching Rest Data Investigation
***

## Table of Contents
***

## Aim <a class="anchor" id="aim"></a>
***

The aim of this notebook is to do some Exploratory data analysis on the data obtained from the Caching Rest Application

## Code Setup <a class="anchor" id="code-setup"></a>
***

### Import Libraries

In [2]:
import pymysql
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper")

### Constants

In [3]:
db_connection = pymysql.connect("pidb","dbuser","password","cachetest" )

### Useful Functions

### Table Read

In [4]:
def read_full_db_table(table_name):
    return pd.read_sql('SELECT * FROM ' + table_name, con=db_connection)

### Query to Dataframe

In [5]:
def obtain_df_from_query(query_string):
    return pd.read_sql(query_string, con=db_connection)

### Cache Hit Bytes to Boolean

In [6]:
def convert_cache_hit_to_boolean(df):
    return df.cache_hit.map({b'\x00' : False, b'\x01' : True})

### DF ids to comma seperated string

In [7]:
def df_ids_to_string(df):
    id_list = df.id.tolist()
    id_list = [str(i) for i in id_list]
    return ''.join(id_list)

## Data Import <a class="anchor" id="data-import"></a>
***

In [8]:
batch_df = read_full_db_table("batch")
batch_df.head()

,id,cache_size_mb,cache_type,end_date,eviction_policy,start_date
0,1,256,ehcache,2018-04-13 14:03:39,LRU,2018-04-13 11:30:05
1,2,512,ehcache,2018-04-14 00:33:22,LRU,2018-04-13 18:48:15
2,3,128,ehcache,2018-04-14 14:06:03,LRU,2018-04-14 12:39:39
3,4,1024,ehcache,2018-04-15 06:37:25,LRU,2018-04-14 19:00:22
4,5,512,hazelcast,2018-04-15 21:27:42,LRU,2018-04-15 15:40:38


## Ehcache Data

In [9]:
ehcache_batch_df = batch_df.query("cache_type == 'ehcache'")
ehcache_batch_df.head()

,id,cache_size_mb,cache_type,end_date,eviction_policy,start_date
0,1,256,ehcache,2018-04-13 14:03:39,LRU,2018-04-13 11:30:05
1,2,512,ehcache,2018-04-14 00:33:22,LRU,2018-04-13 18:48:15
2,3,128,ehcache,2018-04-14 14:06:03,LRU,2018-04-14 12:39:39
3,4,1024,ehcache,2018-04-15 06:37:25,LRU,2018-04-14 19:00:22


### EhCache LRU 128 MB

#### EhCache LRU 128 MB Batch

In [10]:
ehcache_128_LRU_batch_df = ehcache_batch_df.query("cache_size_mb == '128' and eviction_policy == 'LRU'")
ehcache_128_LRU_batch_df.head()

,id,cache_size_mb,cache_type,end_date,eviction_policy,start_date
2,3,128,ehcache,2018-04-14 14:06:03,LRU,2018-04-14 12:39:39


#### EhCache LRU 128 MB Cache Performance

In [11]:
ehcache_128_LRU_cperformance_df = obtain_df_from_query("SELECT * FROM cache_performance cp where cp.batch_id in ("
                                              + df_ids_to_string(ehcache_128_LRU_batch_df)
                                              + ")")
# Issue with Booleans coming in as bytes and not booleans
ehcache_128_LRU_cperformance_df.cache_hit = convert_cache_hit_to_boolean(ehcache_128_LRU_cperformance_df)
# ehcache_128_LRU_cperformance_df = ehcache_128_LRU_cperformance_df.query("cache_hit")
ehcache_128_LRU_cperformance_df.head()

,id,cache_hit,elapsed_time_inns,end_date,file_name,file_sizekb,image_hash,start_date,batch_id
0,32417,False,1639566293,2018-04-14 12:45:43,../images/burger/google-image(0429).jpeg,402,a0e1db15f1aed841398021575ba7b0bc,2018-04-14 12:45:42,3
1,32418,False,1056532664,2018-04-14 12:45:44,../images/cache/google-image(0450).jpeg,41,7a3417c6d2fed9615ece878d32dc1f97,2018-04-14 12:45:43,3
2,32419,False,1101425538,2018-04-14 12:45:45,../images/jumpers/google-image(0225).jpeg,248,b199783399147d5c051cee168fff2648,2018-04-14 12:45:44,3
3,32420,False,1052317559,2018-04-14 12:45:46,../images/dogs/google-image(0056).jpeg,71,bb239b4e9b36007efb93e785b0ef6ebb,2018-04-14 12:45:45,3
4,32421,False,1050180165,2018-04-14 12:45:48,../images/cache/google-image(0615).jpeg,17,fdd44056424886e9fe9c8569ac4c9762,2018-04-14 12:45:46,3


#### EhCache LRU 128 MB Cache Initial Content

In [12]:
ehcache_128_LRU_cinitial_content_df = obtain_df_from_query("SELECT * FROM cache_initial_content cic where cic.batch_id in ("
                                              + df_ids_to_string(ehcache_128_LRU_batch_df)
                                              + ")")
ehcache_128_LRU_cinitial_content_df.head()

,id,file_name,file_sizekb,image_hash,batch_id
0,5006,../images/animals/google-image(0376).jpeg,66,1d76de3e83e4400a226520c45b10db5c,3
1,5007,../images/animals/google-image(0294).jpeg,68,3913378a350367deec4f7b754f7b3074,3
2,5008,../images/animals/google-image(0213).jpeg,118,ae7cfb1ab1945836c2925e43061d7827,3
3,5009,../images/animals/google-image(0219).jpeg,53,8b70b4e1ec8e5a05874d08f0f48133a4,3
4,5010,../images/animals/google-image(0079).jpeg,45,4565da9f4eb917b6ecaf854d22c40147,3


#### EhCache LRU 128 MB Cache Remainder

In [13]:
ehcache_128_LRU_cremainder_df = obtain_df_from_query("SELECT * FROM cache_remainder cr where cr.batch_id in ("
                                              + df_ids_to_string(ehcache_128_LRU_batch_df)
                                              + ")")
ehcache_128_LRU_cremainder_df.head()

,id,file_name,file_sizekb,image_hash,batch_id
0,5573,../images/animals/google-image(0320).jpeg,399,906a05f99aaa6cdc4cad556ef9786f2d,3
1,5574,../images/animals/google-image(0356).png,457,2ce104af6c1a409f0d9554149e998240,3
2,5575,../images/animals/google-image(0014).jpeg,447,634e07f9d076196431a0a7d9b66b893,3
3,5576,../images/animals/google-image(0013).jpeg,334,3a6e82096b37fc830c0d3bf05ecdc971,3
4,5577,../images/animals/google-image(0427).jpeg,174,4a717a437ae230f03845184fb60659dd,3
